# Russian language dataset

In [1]:
from fastai.basics import *
from multiprocessing import Pool
import regex as re
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException

In [2]:
librusec = '/home/u/nas/librusec/lib.rus.ec'
tmp = './tmp/fb2'
data = '../dataset'

In [3]:
!mkdir {data}
!mkdir tmp
!mkdir tmp/fb2
!mkdir tmp/txt

mkdir: cannot create directory ‘tmp’: File exists
mkdir: cannot create directory ‘tmp/fb2’: File exists
mkdir: cannot create directory ‘tmp/txt’: File exists


### unpack zips

In [5]:
zips = get_files(librusec, '.zip')

In [6]:
def unpack(fn):
    !unzip {fn} -d {tmp} >>/dev/null

In [ ]:
for _ in progress_bar(Pool(64).imap_unordered(unpack, zips), len(zips)):
    pass

### convert fb2 to txt

In [7]:
fbs = get_files(tmp, '.fb2')

In [8]:
def convert_fb2(fn):
    !xsltproc FB2_2_txt.xsl {fn} > {str(fn).replace('.fb2','.txt').replace('/fb2','/txt')} 2>>/dev/null

In [ ]:
for _ in progress_bar(Pool(64).imap_unordered(convert_fb2, fbs), len(fbs)):
    pass

### filter txts

In [9]:
txts = get_files('./tmp/txt', '.txt')

In [13]:
len(txts)

428809

In [101]:
# test
fn = txts[0]
match = re.compile('(.|\s)\\1\\1+')
with open(fn, 'r') as f:
    lines = f.read()
lines = 'asdf aaaa dddddfffff' + lines
lines = match.sub(r'\1'*3, lines)             
lines[:1000]

'asdf aaa dddfffУбийцы (Мир пауков - 6)\nКолин Уилсон\n\n\nУилсон Колин\n\nУбийцы (Мир пауков - 6)\n\n\nКолин Уилсон \n\nмир пауков \n\nфантастический роман \n\n"УБИЙЦЫ" \n\nперевод с английского АЛЕКСАНДРА ШАБРИНА \n\nНезадолго до рассвета Найл проснулся от холода - такого пронизывающего, что вспомнилась ночь в пустыне. Простыни сбились вокруг лица, а от дыхания на одеяле образовался кружок влаги. Эту комнату Найл выбрал потому, что она выходила окнами на восток, а ему нравилось просыпаться вместе с солнцем. Сегодня солнца не было. Рассвет проплавлялся сквозь серую туманную пелену, пока комнату не залил холодный, ровный свет. И птиц не было слышно. \n\nЧто-то в этой странной тишине настораживало. Найл, ступая по мягкому шерстяному половику, прошел по комнате к окну, и глазам неожиданно открылся белый пейзаж. Белые крыши, словно горные вершины терялись в перламутрово-сером небе, и огромная площадь была как ковром покрыта той же безликой белизной. Даже вон углы оконных рам прихватило: с

In [102]:
# test
detect(lines)

'ru'

In [104]:
# test
expr = re.compile('([^\n]{150,})([.] )([^\n]{150,})')
while expr.search(lines):
    lines = expr.sub(r'\g<1>.\n\g<3>', lines, 1)
lines[:1000]

'asdf aaa dddfffУбийцы (Мир пауков - 6)\nКолин Уилсон\n\n\nУилсон Колин\n\nУбийцы (Мир пауков - 6)\n\n\nКолин Уилсон \n\nмир пауков \n\nфантастический роман \n\n"УБИЙЦЫ" \n\nперевод с английского АЛЕКСАНДРА ШАБРИНА \n\nНезадолго до рассвета Найл проснулся от холода - такого пронизывающего, что вспомнилась ночь в пустыне. Простыни сбились вокруг лица, а от дыхания на одеяле образовался кружок влаги.\nЭту комнату Найл выбрал потому, что она выходила окнами на восток, а ему нравилось просыпаться вместе с солнцем. Сегодня солнца не было. Рассвет проплавлялся сквозь серую туманную пелену, пока комнату не залил холодный, ровный свет. И птиц не было слышно. \n\nЧто-то в этой странной тишине настораживало. Найл, ступая по мягкому шерстяному половику, прошел по комнате к окну, и глазам неожиданно открылся белый пейзаж.\nБелые крыши, словно горные вершины терялись в перламутрово-сером небе, и огромная площадь была как ковром покрыта той же безликой белизной. Даже вон углы оконных рам прихватило:

In [ ]:
# this will take time, bcs langdetect fails on multithreading
match = re.compile('(.|\s)\\1\\1+')
for fn in progress_bar(txts):
    with open(fn, 'r') as f:
        lines = f.read()
        lines = match.sub(r'\1'*3, lines)
        try:
            if detect(lines) == 'ru':
                with open(f'{data}/{fn.name}', 'w') as c:
                    c.write(lines)
        except LangDetectException as e:
            pass

### concat for vocab

In [ ]:
txts = get_files(data, '.txt')

In [ ]:
sz=0
with open('./tmp/russian_corpus_for_vocab.txt', 'w') as c:
    for fn in txts:
        with open(fn, 'r') as f:
            sz += c.write(f.read() + '\n')
            if sz > 1e+9:
                break